In [1]:
import os

base_path = "/wav2vec2/s-kr/fine-tune/dataset"
kspon_dir = os.path.join(base_path, "KsponSpeech")
kss_dir = os.path.join(base_path, "kss")
train_zeroth_dir = os.path.join(base_path, "zeroth/train_data")
test_zeroth_dir = os.path.join(base_path, "zeroth/test_data")

# 학습 데이터셋 구축
- KpsonSpeech 데이터셋 내 모든 텍스트 데이터 활용

In [2]:
from kspon_preprocess import special_filter, bracket_filter
from glob import glob

kspon_texts = glob(os.path.join(kspon_dir, '**', '*.txt'), recursive=True)
kss_texts = glob(os.path.join(kss_dir, '**', '*.txt'), recursive=True)
train_zeroth_texts = glob(os.path.join(train_zeroth_dir, '**', '*.txt'), recursive=True)
test_zeroth_texts = glob(os.path.join(test_zeroth_dir, '**', '*.txt'), recursive=True)

text_path_list = kspon_texts + kss_texts + train_zeroth_texts + test_zeroth_texts
len(text_path_list) # kpson 텍스트 데이터는 총 612,422 개이다.

648019

In [18]:
# a = "하얗다 파랗다 착하다 나쁘다 성실하다 노력하다"
# # okt = Okt()
# mecab.pos(a)

In [17]:
from tqdm import tqdm
from konlpy.tag import Okt, Mecab
import re

'''
온갖 특수 기호 및 영어, 숫자, 퍼센트 등 한글 외의 데이터는 무시한다.
'''
mecab = Mecab()
remove_re = '[a-zA-Z0-9%]'
raw_texts = list()
for texts in tqdm(text_path_list):
    try:
        with open(texts, 'r', encoding='cp949') as f:
            text = f.read()
    except:
        with open(texts, 'r') as f:
            text = f.read()
    text = special_filter(bracket_filter(text))
    if re.findall(remove_re, text) == []:
        text = mecab.pos(text)
        for tt in text:
            if (tt[1].find("N") == 0) or (tt[1].find("V") == 0):
                raw_texts.append(tt[0])

print(len(raw_texts))
print(raw_texts[:5])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 648019/648019 [04:46<00:00, 2265.39it/s]

5304991
['가', '가', '내', '그것', '고민']


In [23]:
raw_text = list(set(raw_texts))
print(len(raw_text))
print(raw_text[:5])

54156
['합쳤', '무념무상', '무서워요', '그겁니다', '참모습']


# FastText 테스트

In [24]:
from gensim.models import FastText

# results = [line.split(" ") for line in tqdm(raw_texts)]
results = raw_text
len(results)

54156

In [25]:
ft_model = FastText(results, window=3, min_count=2, workers=4, sg=1)

In [65]:
test_data

'그 바람에 나의 몸도 겹처 쓰러지며 한창 피어난 노란 동백꼿 속으로 폭 파뭍혀 버렸다'

In [66]:
test_data = "그 바람에 나의 몸도 겹처 쓰러지며 한창 피어난 노란 동백꼿 속으로 폭 파뭍혀 버렸다"

num = 0
ch_list = list()
test_pos = mecab.pos(test_data)
pos_list = [pos for pos in test_pos if (pos[1].find("N") == 0) or (pos[1].find("V") == 0)]

for i, data1 in enumerate(tqdm(pos_list)):
    total_score = 0.0
    for data2 in pos_list:
        if data1[0] != data2[0]:
            # sim_score = ft_model.wv.similarity(data1[0], data2[0])
            sim_score = ft_model.wv.relative_cosine_similarity(data1[0], data2[0])
            # print(sim_score)
            total_score += sim_score
    avg_score = total_score / (len(pos_list)-1)
    print(f'{i+1}번째 단어 "{data1[0]}"의 평균 유사도: {avg_score}')
    if avg_score < 0.1:
        ch_list.append(data1[0])
        # print(f'오류 단어 "{data1[0]}"의 평균 유사도: {avg_score}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 97.21it/s]

1번째 단어 "바람"의 평균 유사도: 0.022029044713533976
2번째 단어 "나"의 평균 유사도: 0.03132383064248845
3번째 단어 "몸"의 평균 유사도: 0.03595743404316869
4번째 단어 "겹"의 평균 유사도: 0.03284787524439991
5번째 단어 "처"의 평균 유사도: 0.03501451858287186
6번째 단어 "쓰러지"의 평균 유사도: -0.2217519540661151
7번째 단어 "피어난"의 평균 유사도: 0.014342176409413604
8번째 단어 "노란"의 평균 유사도: 0.014915885617335383
9번째 단어 "동백"의 평균 유사도: -0.14374449211199664
10번째 단어 "꼿"의 평균 유사도: 0.005966133853096392
11번째 단어 "속"의 평균 유사도: 0.03213959350013029
12번째 단어 "폭"의 평균 유사도: 0.030789090125959654
13번째 단어 "파"의 평균 유사도: 0.02891276903067962
14번째 단어 "뭍"의 평균 유사도: 0.3552337380271875
15번째 단어 "버렸"의 평균 유사도: 0.009214400145787607


In [67]:
ch_list

['바람', '나', '몸', '겹', '처', '쓰러지', '피어난', '노란', '동백', '꼿', '속', '폭', '파', '버렸']

In [55]:
ch_list

['바람', '쓰러지', '피어난', '노란', '동백', '꼿', '뭍', '버렸']

In [57]:
from soynlp.hangle import jamo_levenshtein
import itertools

score_list = list()
results_test = list(itertools.chain(*results))
for re in tqdm(results_test):
    for ch in ch_list:
        if re != ch:
            score = jamo_levenshtein(re, ch)
            # print(score)
            if score < 1.0:
                score_list.append([ch, re, score])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 140001/140001 [01:09<00:00, 2001.87it/s]


In [58]:
score_list

[['뭍', '무', 0.3333333333333333],
 ['뭍', '무', 0.3333333333333333],
 ['뭍', '무', 0.3333333333333333],
 ['꼿', '모', 0.6666666666666666],
 ['뭍', '모', 0.6666666666666666],
 ['뭍', '출', 0.6666666666666666],
 ['꼿', '로', 0.6666666666666666],
 ['뭍', '우', 0.6666666666666666],
 ['뭍', '주', 0.6666666666666666],
 ['꼿', '첫', 0.6666666666666666],
 ['꼿', '핫', 0.6666666666666666],
 ['꼿', '콘', 0.6666666666666666],
 ['꼿', '모', 0.6666666666666666],
 ['뭍', '모', 0.6666666666666666],
 ['꼿', '코', 0.6666666666666666],
 ['뭍', '망', 0.6666666666666666],
 ['꼿', '꺼', 0.6666666666666666],
 ['뭍', '눌', 0.6666666666666666],
 ['꼿', '모', 0.6666666666666666],
 ['뭍', '모', 0.6666666666666666],
 ['뭍', '만', 0.6666666666666666],
 ['꼿', '손', 0.6666666666666666],
 ['꼿', '랫', 0.6666666666666666],
 ['뭍', '머', 0.6666666666666666],
 ['꼿', '솔', 0.6666666666666666],
 ['꼿', '끊', 0.6666666666666666],
 ['뭍', '주', 0.6666666666666666],
 ['꼿', '소', 0.6666666666666666],
 ['꼿', '갯', 0.6666666666666666],
 ['꼿', '짓', 0.6666666666666666],
 ['뭍', '부'

In [29]:
test_text = ' '.join(results[0])
test_text

'엉화 영화 후대폰 휴대폰 가긴 가야지 그니까 인까 내가 그것도 고민을 군대도 고민을 하는 이유도 그엇 알티도 있다고 했잖아 내 아까 그래서 웬만하면 알티로 군대를 가고 싶은데 그걸 못한다고 하니까 그거에 대해서도 고민이 많은 거고'

In [64]:
change_list = list()
pos_data = [pos for pos in mecab.pos(test_data) if (pos[1].find("N") == 0) or (pos[1].find("V") == 0)]
pos_data = list(itertools.chain(*pos_data))
pos_data = pos_data[0::2]

for score_word in score_list:
    test_text = pos_data.copy()
    total_score = 0.0
    origin = score_word[0]
    # print(origin)
    related = score_word[1]
    result_idx = test_text.index(origin)
    test_text[result_idx] = related
    for text in test_text:
        sim_score = ft_model.wv.similarity(related, text)
        total_score += sim_score
    avg_score = total_score / len(test_text)
    # print(f'{origin} -> {related} 평균 유사도: {avg_score}')
    if avg_score > 0.45:
        change_list.append([origin, related])
        print(f'{origin} -> {related} 평균 유사도: {avg_score}')

꼿 -> 첫 평균 유사도: 0.4624132111668587
꼿 -> 손 평균 유사도: 0.45428466704518844
꼿 -> 갯 평균 유사도: 0.45519688179095585
꼿 -> 짓 평균 유사도: 0.45601183076699575
꼿 -> 몸 평균 유사도: 0.4626840693255266
뭍 -> 몸 평균 유사도: 0.47843348756432535
꼿 -> 곤 평균 유사도: 0.45034836083650587
꼿 -> 손 평균 유사도: 0.45428466704518844
꼿 -> 꿍 평균 유사도: 0.4558863701919715
뭍 -> 꿍 평균 유사도: 0.4716410882771015
꼿 -> 꿍 평균 유사도: 0.4558863701919715
뭍 -> 꿍 평균 유사도: 0.4716410882771015
뭍 -> 막 평균 유사도: 0.4697407342803975
뭍 -> 밌 평균 유사도: 0.4696086949358384
뭍 -> 웁 평균 유사도: 0.4718762807548046
꼿 -> 또 평균 유사도: 0.45222194915016495
꼿 -> 꽂 평균 유사도: 0.45108116529881953
뭍 -> 몽 평균 유사도: 0.46058764110008876
뭍 -> 꿈 평균 유사도: 0.4607813843836387
뭍 -> 맥 평균 유사도: 0.46475623672207195
뭍 -> 뿐 평균 유사도: 0.46753716220458347
꼿 -> 꺾 평균 유사도: 0.45361163914203645
뭍 -> 멀 평균 유사도: 0.45319853809972604
꼿 -> 뺏 평균 유사도: 0.45439228328565756
꼿 -> 꽂 평균 유사도: 0.45108116529881953
꼿 -> 뒷 평균 유사도: 0.45781302377581595
뭍 -> 맺 평균 유사도: 0.4726488043864568
꼿 -> 몸 평균 유사도: 0.4626840693255266
뭍 -> 몸 평균 유사도: 0.478433487564325

In [62]:
test_text

[('바람', 'NNG'),
 ('나', 'NP'),
 ('몸', 'NNG'),
 ('겹', 'NNG'),
 ('처', 'NNG'),
 ('쓰러지', 'VV'),
 ('피어난', 'VV+ETM'),
 ('노란', 'VA+ETM'),
 ('동백', 'NNG'),
 ('꼿', 'NNG'),
 ('속', 'NNG'),
 ('폭', 'NNG'),
 ('파', 'NNG'),
 ('뭍', 'NNG'),
 ('버렸', 'VX+EP')]

# 동시등장행렬

In [7]:
from tqdm import tqdm
from konlpy.tag import Okt, Mecab
import re

'''
vocab에 입력할 단어 생성
- 온갖 특수 기호 및 영어, 숫자, 퍼센트 등 한글 외의 데이터는 무시한다.
'''
mecab = Mecab()
remove_re = '[a-zA-Z0-9%]'
raw_texts = list()
text_list = list()
for texts in tqdm(text_path_list):
    try:
        with open(texts, 'r', encoding='cp949') as f:
            text = f.read()
    except:
        with open(texts, 'r') as f:
            text = f.read()
    text = special_filter(bracket_filter(text))
    if re.findall(remove_re, text) == []:
        text_list.append(text)
        text = mecab.pos(text)
        for tt in text:
            if (tt[1].find("N") == 0) or (tt[1].find("V") == 0):
                raw_texts.append(tt[0])

print(len(raw_texts))
print(raw_texts[:5])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 648019/648019 [04:40<00:00, 2307.10it/s]

5304991
['가', '가', '내', '그것', '고민']


In [5]:
raw_words = list(set(raw_texts))

print(len(raw_words))
print(raw_words[:5])

54156
['힌두교', '진이', '보험증', '초한지', '효정']


In [118]:
import json

def build_vocabulary(word_list:list) -> list:
    '''
    리스트에 포함된 모든 단어들을 dict 형태의 vocab으로 변환
    '''
    word_list.sort()
    
    vocab_dict = {}
    for index, word in enumerate(word_list):
        vocab_dict[word] = index
        
    return vocab_dict


# vocab_dict = build_vocabulary(raw_words)

vocab_path = "/wav2vec2/s-kr/fine-tune/transformer/post_vocab.json"

with open(vocab_path, 'r') as f:
    origin_vocab_dict = json.load(f)
# with open(vocab_path, 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

print(len(origin_vocab_dict))

54156


In [5]:
import pandas as pd
import numpy as np

co_ocurrence_vectors = pd.DataFrame(
    np.zeros([len(origin_vocab_dict), len(origin_vocab_dict)]),
    index = origin_vocab_dict.keys(),
    columns = origin_vocab_dict.keys()
)

co_ocurrence_vectors.head()

,怨,移,議,가,가가,가각,가감,가거,가게,가겐,...,힘줘,힘줘서,힘차,힘찬,힙,힙스터,힙합,힝,留,理
怨,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
移,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
議,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가가,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# test_df = co_ocurrence_vectors.copy()

In [ ]:
# def build_context(page:str, co_ocurrence_vectors: pd.DataFrame) -> pd.DataFrame:
#     '''
#     Updates co-ocurrence vectors based on
#     text read from the page.
#     '''
#     for index, element in enumerate(page):
#         # Build start and finish of context
#         start = 0 if index-2 < 0 else index-2
#         finish = len(page) if index+2 > len(page) else index+3
#         # Retrieve Context for word
#         context = page[start:index]+page[index+1:finish]
#         for word in context:
#             # Update Co-Occurrence Matrix 
#             co_ocurrence_vectors.loc[element, word] = (co_ocurrence_vectors.loc[element, word]+1)
            
#     return co_ocurrence_vectors


# co_ocurrence_vectors = build_context(usa_article_token, co_ocurrence_vectors)

In [40]:
page = "울타리가 되야하는 가정이 문너지고 있는 건 아닌지 걱정되는 소식 하나 더 전해드립니다"

elements_list = [elements for elements in mecab.pos(page) if (elements[1].find("N") == 0) or (elements[1].find("V") == 0)]
for idx1, element1 in enumerate(elements_list):
    for idx2, element2 in enumerate(elements_list):
        if idx1 < idx2:
            try:
                test_df.loc[element1[0], element2[0]] = (test_df.loc[element1[0], element2[0]]+1)
                print(element1[0], element2[0], test_df.loc[element1[0], element2[0]])
            except:
                pass

In [8]:
pages = text_list

## 11~12시간 소요.. ㄷㄷ(vocab 54000개 기준)
for page in tqdm(pages):
    elements_list = [elements for elements in mecab.pos(page) if (elements[1].find("N") == 0) or (elements[1].find("V") == 0)]
    for idx1, element1 in enumerate(elements_list):
        for idx2, element2 in enumerate(elements_list):
            # if idx1 < idx2:
                try:
                    co_ocurrence_vectors.loc[element1[0], element2[0]] = (co_ocurrence_vectors.loc[element1[0], element2[0]]+1)
                except:
                    pass

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 640832/640832 [11:36:08<00:00, 15.34it/s]


In [9]:
co_ocurrence_vectors.head()

,怨,移,議,가,가가,가각,가감,가거,가게,가겐,...,힘줘,힘줘서,힘차,힘찬,힙,힙스터,힙합,힝,留,理
怨,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
移,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
議,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
가,0.0,0.0,0.0,84753.0,22.0,1.0,1.0,6.0,188.0,2.0,...,0.0,0.0,0.0,1.0,13.0,0.0,28.0,0.0,0.0,0.0
가가,0.0,0.0,0.0,22.0,77.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
co_ocurrence_vectors.to_csv("/wav2vec2/s-kr/fine-tune/transformer/co_occur_df.csv")

In [39]:
co_ocurrence_vectors.loc['겹쳐'].sort_values(ascending=False).head(50)

겹쳐     62.0
가지     43.0
있      26.0
거      22.0
나      19.0
하      14.0
그게     14.0
이      13.0
보      12.0
그거     12.0
내      11.0
일      10.0
가      10.0
했       9.0
같       9.0
때       9.0
게       8.0
우리      8.0
겹치      7.0
이게      7.0
말       7.0
아니      7.0
달       7.0
해       7.0
야       6.0
좋       6.0
날짜      6.0
되       6.0
지       6.0
삼       6.0
그때      6.0
그래      5.0
수       5.0
주       5.0
개       5.0
돼       5.0
몸       5.0
걔       5.0
때문      4.0
노란      4.0
속       4.0
맞       4.0
폭       4.0
파묻혀     4.0
쓰러지     4.0
판       4.0
버렸      4.0
시간      4.0
피어난     4.0
엄마      4.0
Name: 겹쳐, dtype: float64

In [ ]:
co_ocurrence_vectors = pd.read_csv("/wav2vec2/s-kr/fine-tune/transformer/co_occur_df.csv")
co_ocurrence_vectors.head()

# 동시등장빈도에 따른 코사인 유사도

In [359]:
import pandas as pd
import numpy as np

input_text = "울터리가 되야하는 가정이 무너지고 있는 건 아닌지 걱정되는 소식 하나 더 전해드립니다"
input_elements = [elements[0] for elements in mecab.pos(input_text) if (elements[1].find("N") == 0) or (elements[1].find("V") == 0)]
vocab_dict = build_vocabulary(input_elements)

input_co_ocur_vector = pd.DataFrame(
    np.zeros([len(vocab_dict), len(vocab_dict)]),
    index = vocab_dict.keys(),
    columns = vocab_dict.keys()
)

input_co_ocur_vector

,가정,걱정,건,되,드립니다,무너지,소식,아닌지,있,전해,터리,하,하나
가정,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
걱정,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
건,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
되,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
드립니다,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
무너지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
소식,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
아닌지,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
있,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
전해,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [360]:
'''
입력받은 문장에 대한 동시등장빈도 확인
'''
for element1 in input_elements:
    for element2 in input_elements:
        try:
            input_co_ocur_vector.loc[element1, element2] = co_ocurrence_vectors.loc[element1, element2]
            # print(input_co_ocur_vector.loc[element1, element2])
        except:
            input_co_ocur_vector.loc[element1, element2] = 0

input_co_ocur_vector

,가정,걱정,건,되,드립니다,무너지,소식,아닌지,있,전해,터리,하,하나
가정,531.0,0.0,13.0,31.0,0.0,0.0,0.0,0.0,197.0,0.0,0.0,151.0,12.0
걱정,0.0,2121.0,78.0,514.0,0.0,1.0,0.0,2.0,536.0,1.0,0.0,758.0,64.0
건,13.0,78.0,11013.0,1201.0,0.0,2.0,4.0,5.0,3616.0,1.0,1.0,3610.0,235.0
되,31.0,514.0,1201.0,53435.0,5.0,11.0,8.0,9.0,14428.0,8.0,4.0,16029.0,1247.0
드립니다,0.0,0.0,0.0,5.0,42.0,0.0,0.0,0.0,12.0,1.0,0.0,12.0,1.0
무너지,0.0,1.0,2.0,11.0,0.0,57.0,0.0,0.0,18.0,0.0,0.0,23.0,3.0
소식,0.0,0.0,4.0,8.0,0.0,0.0,263.0,0.0,82.0,8.0,0.0,39.0,2.0
아닌지,0.0,2.0,5.0,9.0,0.0,0.0,0.0,70.0,25.0,0.0,0.0,20.0,2.0
있,197.0,536.0,3616.0,14428.0,12.0,18.0,82.0,25.0,194700.0,19.0,6.0,42240.0,4266.0
전해,0.0,1.0,1.0,8.0,1.0,0.0,8.0,0.0,19.0,121.0,0.0,17.0,0.0


In [363]:
from numpy import dot
from numpy.linalg import norm

# 코사인 유사도를 구하는 함수
def cos_sim(a, b):
    try:
        if (a == [.0] * len(a)) or (b == [.0] * len(b)):
            raise RuntimeWarning
        cosine = dot(a, b)/(norm(a)*norm(b))
    except:
        if (a == [.0] * len(a)) or (b != [.0] * len(b)):
            cosine = dot(a, b)/((norm(a)+0.01)*(norm(b)))
        elif (a != [.0] * len(a)) or (b == [.0] * len(b)):
            cosine = dot(a, b)/(norm(a)*(norm(b)+0.01))
        else:
            cosine = dot(a, b)/((norm(a)+0.01)*(norm(b)+0.01))
    finally:
        return cosine

typo_word = list()
input_co_occur_list = input_co_ocur_vector.values.tolist()
for idx, input_list1 in enumerate(input_co_occur_list):
    total_cosine = 0.0
    for input_list2 in input_co_occur_list:
        if input_list1 != input_list2:
            total_cosine += cos_sim(input_list1, input_list2)

    avg_cosine = total_cosine / (len(input_list1) - 1)
    
    # print(f'단어 "{input_co_ocur_vector.index[idx]}" 에 대한 평균 코사인 유사도: {avg_cosine}')
    ## 이 부분에서 임계치 설정 필요
    if avg_cosine < 0.30:
        typo_word.append(input_co_ocur_vector.index[idx])
        print(f'단어 "{input_co_ocur_vector.index[idx]}" 에 대한 평균 코사인 유사도: {avg_cosine}')

단어 "가정" 에 대한 평균 코사인 유사도: 0.19878994386848667
단어 "걱정" 에 대한 평균 코사인 유사도: 0.21647661804648988
단어 "건" 에 대한 평균 코사인 유사도: 0.22530904973809063
단어 "되" 에 대한 평균 코사인 유사도: 0.26700994863522304
단어 "드립니다" 에 대한 평균 코사인 유사도: 0.18883214305360516
단어 "무너지" 에 대한 평균 코사인 유사도: 0.23124000959720262
단어 "소식" 에 대한 평균 코사인 유사도: 0.15336430715486962
단어 "아닌지" 에 대한 평균 코사인 유사도: 0.21164931588278102
단어 "전해" 에 대한 평균 코사인 유사도: 0.11660483320729163
단어 "터리" 에 대한 평균 코사인 유사도: 0.17789800852640336
단어 "하나" 에 대한 평균 코사인 유사도: 0.2104224947821235


In [364]:
typo_word

['가정', '걱정', '건', '되', '드립니다', '무너지', '소식', '아닌지', '전해', '터리', '하나']

In [367]:
from soynlp.hangle import jamo_levenshtein

ed_dict = dict()
vocab_list = list(origin_vocab_dict.keys())
for typo in tqdm(typo_word):  # vocab 54000개 기준 오탈자 개당 10초 소요
    ed_list = list()
    for vocab in vocab_list:
        ed_score = jamo_levenshtein(typo, vocab)
        if ed_score < 0.35:
            ed_list.append(vocab)
    ed_dict[typo] = ed_list
            # print(f'오탈자 "{typo}" 와 유사한 사전 단어: {vocab} - {ed_score}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [01:36<00:00,  8.78s/it]


In [368]:
ed_dict

{'가정': ['가성',
  '가장',
  '가저',
  '가전',
  '가점',
  '가정',
  '가중',
  '가징',
  '감정',
  '강정',
  '개정',
  '계정',
  '고정',
  '과정',
  '교정',
  '구정',
  '규정',
  '그정',
  '기정',
  '나정',
  '다정',
  '사정',
  '자정',
  '하정'],
 '걱정': ['걱정', '검정', '격정', '국정', '덕정', '석정', '적정'],
 '건': ['간',
  '갠',
  '갼',
  '걘',
  '거',
  '건',
  '걷',
  '걸',
  '검',
  '겁',
  '것',
  '겄',
  '겉',
  '겐',
  '견',
  '곤',
  '관',
  '군',
  '권',
  '귄',
  '균',
  '근',
  '긴',
  '껀',
  '넌',
  '던',
  '떤',
  '런',
  '먼',
  '번',
  '뻔',
  '선',
  '썬',
  '언',
  '전',
  '쩐',
  '천',
  '컨',
  '턴',
  '펀',
  '헌'],
 '되': ['괴',
  '꾀',
  '뇌',
  '다',
  '대',
  '댜',
  '더',
  '데',
  '뎌',
  '도',
  '돼',
  '되',
  '된',
  '될',
  '됨',
  '됬',
  '두',
  '둬',
  '뒤',
  '듀',
  '드',
  '듸',
  '디',
  '뙤',
  '뢰',
  '뵈',
  '쇠',
  '쐬',
  '외',
  '죄',
  '쬐',
  '최',
  '퇴',
  '푀',
  '회'],
 '드립니다': ['드립니다', '들립니다'],
 '무너지': ['무너져', '무너지', '무너진', '무너질', '무너짐'],
 '소식': ['고식',
  '노식',
  '도식',
  '보식',
  '사식',
  '서식',
  '소닉',
  '소속',
  '소시',
  '소식',
  '소신',
  '소실',
  '소심',
  '수식',
  '시식',
  '조식',


In [369]:
results_list = list()
for origin_word in list(ed_dict.keys()):
    og_words = list()
    replace_words = list()
    cosine_value = list()
    
    for ed_word in ed_dict[origin_word]:
        ed_vector = input_co_ocur_vector.rename(columns={origin_word: ed_word}, index={origin_word: ed_word}, inplace=False)
        ed_vector = pd.DataFrame(
            np.zeros([len(ed_vector), len(ed_vector)]),
            index = ed_vector.index,
            columns = ed_vector.columns
        )
        
        for element1 in ed_vector.columns:
            for element2 in ed_vector.columns:
                try:
                    ed_vector.loc[element1, element2] = co_ocurrence_vectors.loc[element1, element2]
                except:
                    ed_vector.loc[element1, element2] = 0
                    
        ed_co_occur_list = ed_vector.values.tolist()
        total_cosine = 0.0
        ed_idx = list(ed_vector.columns).index(ed_word)
        for input_list1 in ed_co_occur_list:
            if ed_co_occur_list[ed_idx] != input_list1:
                total_cosine += cos_sim(ed_co_occur_list[ed_idx], input_list1)

        avg_cosine = total_cosine / (len(ed_co_occur_list[ed_idx]) - 1)
        og_words.append(origin_word)
        replace_words.append(ed_word)
        cosine_value.append(avg_cosine)
    
    cs_idx = cosine_value.index(max(cosine_value))
    rp_word = replace_words[cs_idx]
    results_list.append([origin_word, rp_word])
        # print(f'단어 "{origin_word}" -> "{ed_word}" 으로 변환할 때 평균 코사인 유사도: {avg_cosine}')

In [370]:
results_list

[['가정', '고정'],
 ['걱정', '격정'],
 ['건', '거'],
 ['되', '듸'],
 ['드립니다', '드립니다'],
 ['무너지', '무너질'],
 ['소식', '보식'],
 ['아닌지', '아닌진'],
 ['전해', '선해'],
 ['터리', '터로'],
 ['하나', '해나']]

In [371]:
output_text = input_text
for results in results_list:
    output_text = output_text.replace(results[0], results[1])
    
print(output_text)

울터로가 듸야하는 고정이 무너질고 있는 거 아닌진 격정듸는 보식 해나 더 선해드립니다


# 추론 테스트
- 오탈자 말뭉치를 활용
- 1,000개의 정답-오탈자 데이터셋을 활용
- 수정 요소가 마침표만 있는 경우는 데이터 제외
- 특수기호는 전부 데이터 제외
- 형태소 품사는 명사, 동사, 형용사만

In [235]:
test_dataset_file = "/wav2vec2/s-kr/fine-tune/transformer/Korean_corpus_2021.tsv"
df = pd.read_csv(test_dataset_file, sep='\t', index_col=0)

df.head()

,id,original_form,form,corrected_form
0,ERRW1905000308.26.1,#이마트 트레이더스 가서 장만해온..,#이마트 트레이더스 가서 장만해온..,#이마트 트레이더스 가서 장만해 온…
1,ERRW1905000308.26.2,.,.,.
2,ERRW1905000308.26.3,#비포락토 #유산균 넘나 잘먹는다 우리집 꼬맹들..,#비포락토 #유산균 넘나 잘먹는다 우리집 꼬맹들..,"#비포락토 #유산균 넘나 잘 먹는다, 우리 집 꼬맹이들…"
3,ERRW1905000308.26.4,세상 톡 털어 넣고 아직 어린 둘째도..,세상 톡 털어 넣고 아직 어린 둘째도..,"세상 톡 털어 넣고, 아직 어린 둘째도…"
4,ERRW1905000308.26.5,목걸림 따위 없이 물에 태워먹지 않아도..,목걸림 따위 없이 물에 태워먹지 않아도..,목 걸림 따위 없이 물에 태워 먹지 않아도...


In [236]:
df.drop(['id', 'original_form'], axis=1, inplace=True)
test_df = df.copy()
test_df.head()

,form,corrected_form
0,#이마트 트레이더스 가서 장만해온..,#이마트 트레이더스 가서 장만해 온…
1,.,.
2,#비포락토 #유산균 넘나 잘먹는다 우리집 꼬맹들..,"#비포락토 #유산균 넘나 잘 먹는다, 우리 집 꼬맹이들…"
3,세상 톡 털어 넣고 아직 어린 둘째도..,"세상 톡 털어 넣고, 아직 어린 둘째도…"
4,목걸림 따위 없이 물에 태워먹지 않아도..,목 걸림 따위 없이 물에 태워 먹지 않아도...


In [237]:
import random
import re

re_string1 = '[^A-Za-z0-9가-힣]'

test_df['form'] = test_df['form'].apply(lambda x: re.sub(re_string1, '', str(x)) > 4)
test_df['corrected_form'] = test_df['corrected_form'].apply(lambda x: re.sub(re_string1, '', str(x)))
test_df.head()

,form,corrected_form
0,이마트트레이더스가서장만해온,이마트트레이더스가서장만해온
1,,
2,비포락토유산균넘나잘먹는다우리집꼬맹들,비포락토유산균넘나잘먹는다우리집꼬맹이들
3,세상톡털어넣고아직어린둘째도,세상톡털어넣고아직어린둘째도
4,목걸림따위없이물에태워먹지않아도,목걸림따위없이물에태워먹지않아도


In [332]:
new_df = test_df[test_df['form'] != test_df['corrected_form']]
new_df.head()

,form,corrected_form
2,비포락토유산균넘나잘먹는다우리집꼬맹들,비포락토유산균넘나잘먹는다우리집꼬맹이들
9,지금기획세트있을때쟁겨둘각,지금기획세트있을때쟁여둘각
23,이런게다있네용,이런게다있네요
30,이런게깨알육아장비빨이라는거인정,이런게깨알육아장비발이라는거인정
32,귀욤하기까지해서선물하기도괜찮은데욤,귀염하기까지해서선물하기도괜찮은데요


In [335]:
new_df = new_df.apply(lambda x: x if len(x) > 4 else " ")
new_df = new_df[new_df['form'] != new_df['corrected_form']]
new_df.head()

,form,corrected_form
2,비포락토유산균넘나잘먹는다우리집꼬맹들,비포락토유산균넘나잘먹는다우리집꼬맹이들
9,지금기획세트있을때쟁겨둘각,지금기획세트있을때쟁여둘각
23,이런게다있네용,이런게다있네요
30,이런게깨알육아장비빨이라는거인정,이런게깨알육아장비발이라는거인정
32,귀욤하기까지해서선물하기도괜찮은데욤,귀염하기까지해서선물하기도괜찮은데요


In [336]:
print(len(new_df['form']))
print(len(new_df['corrected_form']))

104777
104777


In [337]:
refine_idx = list(new_df.index)
random.shuffle(refine_idx)
refine_idx = refine_idx[:1000]
refine_idx[:10]

[251570,
 378055,
 392620,
 244854,
 350913,
 534282,
 339986,
 405264,
 474139,
 546971]

In [372]:
refine_df = df.loc[refine_idx]
refine_df = refine_df.reset_index(drop=True)
refine_df.head()

,form,corrected_form
0,완전 좋겟다,완전 좋겠다.
1,나 싱가폴 안가봤는데 너는?,"나 싱가포르 안 가봤는데, 너는?"
2,다 같은맘이군여,다 같은 맘이군요.
3,나는 그냥 볼라고,나는 그냥 보려고.
4,응 ㅋㅋ 나중에 시간날떄,"응, ㅋㅋ 나중에 시간 날 때"


In [373]:
re_string2 = '[^A-Za-z0-9가-힣 ]'

refine_df['form'] = refine_df['form'].apply(lambda x: re.sub(re_string2, '', str(x)))
refine_df['corrected_form'] = refine_df['corrected_form'].apply(lambda x: re.sub(re_string2, '', str(x)))
refine_df.head()

,form,corrected_form
0,완전 좋겟다,완전 좋겠다
1,나 싱가폴 안가봤는데 너는,나 싱가포르 안 가봤는데 너는
2,다 같은맘이군여,다 같은 맘이군요
3,나는 그냥 볼라고,나는 그냥 보려고
4,응 나중에 시간날떄,응 나중에 시간 날 때


In [374]:
refine_df = refine_df[:200]
refine_df.head()

,form,corrected_form
0,완전 좋겟다,완전 좋겠다
1,나 싱가폴 안가봤는데 너는,나 싱가포르 안 가봤는데 너는
2,다 같은맘이군여,다 같은 맘이군요
3,나는 그냥 볼라고,나는 그냥 보려고
4,응 나중에 시간날떄,응 나중에 시간 날 때


In [377]:
import time

def cos_sim(a, b):
    try:
        if (a == [.0] * len(a)) or (b == [.0] * len(b)):
            raise RuntimeWarning
        cosine = dot(a, b)/(norm(a)*norm(b))
    except:
        if (a == [.0] * len(a)) or (b != [.0] * len(b)):
            cosine = dot(a, b)/((norm(a)+0.01)*(norm(b)))
        elif (a != [.0] * len(a)) or (b == [.0] * len(b)):
            cosine = dot(a, b)/(norm(a)*(norm(b)+0.01))
        else:
            cosine = dot(a, b)/((norm(a)+0.01)*(norm(b)+0.01))
    finally:
        return cosine

# 오탈자 셋과 정답 셋을 분리
correct_text = refine_df['corrected_form']
typo_text = refine_df['form']
output_text_list = list()
start = time.time()

for tp_text in tqdm(typo_text):
    ## 오탈자 문장마다 명사, 동사, 형용사 단어만 추출
    input_elements = [elements[0] for elements in mecab.pos(tp_text) if (elements[1].find("N") == 0) or (elements[1].find("V") == 0)]
    ## 각 문장마다 동시등장행렬 생성을 위해 입력받은 문장으로 vocab 생성
    vocab_dict = build_vocabulary(input_elements)
    ## 입력받은 문장에 대한 base 행렬을 생성
    input_co_ocur_vector = pd.DataFrame(
        np.zeros([len(vocab_dict), len(vocab_dict)]),
        index = vocab_dict.keys(),
        columns = vocab_dict.keys()
    )
    ## loc를 활용하여 단어쌍이 등장하는 횟수만큼 1씩 덧셈하며 단어쌍이 vocab 동시등장행렬에 없을 경우는 그냥 0 을 입력
    for element1 in input_elements:
        for element2 in input_elements:
            try:
                input_co_ocur_vector.loc[element1, element2] = co_ocurrence_vectors.loc[element1, element2]
            except:
                input_co_ocur_vector.loc[element1, element2] = 0
    ## 단어별 코사인 유사도를 계산 후 평균값을 계산, 단 같은 값끼리는 의미가 없으므로 != 을 통해 pass 하도록 한다
    typo_word = list()
    input_co_occur_list = input_co_ocur_vector.values.tolist()
    for idx, input_list1 in enumerate(input_co_occur_list):
        if len(input_co_occur_list) > 1:
            total_cosine = 0.0
            for input_list2 in input_co_occur_list:
                if input_list1 != input_list2:
                    total_cosine += cos_sim(input_list1, input_list2)

                # input_list1 에 대한 평균 코사인 유사도를 계산
            avg_cosine = total_cosine / (len(input_list1) - 1)
                # 코사인 유사도 임계치 설정
            if avg_cosine < 0.15:
                # typo_word 에 입력된 단어는 '오탈자'로 확정 인식
                typo_word.append(input_co_ocur_vector.index[idx])
        else:
            pass
        ## 모든 vocab 과 입력받은 문장의 단어끼리 하나하나 편집거리 계산
    ed_dict = dict()
    vocab_list = list(origin_vocab_dict.keys())
    # vocab 54000개 기준 오탈자 개당 10초 소요
    for typo in typo_word:
        ed_list = list()
        for vocab in vocab_list:
            ed_score = jamo_levenshtein(typo, vocab)
            # 편집거리 임계치 설정
            if ed_score < 0.5:
                ed_list.append(vocab)
        # 각 오탈자 별 대체 단어 후보들을 입력
        ed_dict[typo] = ed_list
    ## 이후 오탈자를 대체 단어로 바꿔주면서 동시등장행렬, 코사인 유사도, 편집거리 계산을 동일하게 진행
    results_list = list()
    for origin_word in list(ed_dict.keys()):
        og_words = list()
        replace_words = list()
        cosine_value = list()

        for ed_word in ed_dict[origin_word]:
            # 오탈자 컬럼명을 대체 단어 컬럼명 및 인덱스명으로 교체
            ed_vector = input_co_ocur_vector.rename(columns={origin_word: ed_word}, index={origin_word: ed_word}, inplace=False)
            # 데이터 값들 전부 0으로 리셋
            ed_vector = pd.DataFrame(
                np.zeros([len(ed_vector), len(ed_vector)]),
                index = ed_vector.index,
                columns = ed_vector.columns
            )

            for element1 in ed_vector.columns:
                for element2 in ed_vector.columns:
                    try:
                        ed_vector.loc[element1, element2] = co_ocurrence_vectors.loc[element1, element2]
                    except:
                        ed_vector.loc[element1, element2] = 0

            ed_co_occur_list = ed_vector.values.tolist()
            total_cosine = 0.0
            ed_idx = list(ed_vector.columns).index(ed_word)
            for input_list1 in ed_co_occur_list:
                if ed_co_occur_list[ed_idx] != input_list1:
                    total_cosine += cos_sim(ed_co_occur_list[ed_idx], input_list1)

            avg_cosine = total_cosine / (len(ed_co_occur_list[ed_idx]) - 1)
            # 기존 오탈자, 대체 단서, 평균 코사인 유사도 계산 값을 각각 리스트에 넣는다
            og_words.append(origin_word)
            replace_words.append(ed_word)
            cosine_value.append(avg_cosine)
        if cosine_value != []:
        # 가장 유사도가 높은 값이 오탈자를 대체할 단어로 지정된다
            cs_idx = cosine_value.index(max(cosine_value))
            rp_word = replace_words[cs_idx]
            # 최종적으로 오탈자와 확정된 대체 단어를 리스트에 같이 입력
            results_list.append([origin_word, rp_word])
            
    if results_list != []:
    ## 각 오탈자를 대체 단어로 교체하면서 결과 텍스트를 최종적으로 리스트에 입력한다.
        output_text = tp_text
        for results in results_list:
            output_text = output_text.replace(results[0], results[1])
        output_text_list.append(output_text)
    else:
        output_text_list.append(tp_text)

end = time.time()

print("Time: ", end-start)
print(len(output_text_list))
print(output_text_list[:10])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [1:22:13<00:00, 24.67s/it]

Time:  4933.653675556183
200
['완전 좋겟다', '나 싱가폴 안가봤는데 너는', '다 가은많이귄여', '나는 그냥 볼라고', '응  자중에 시관나떄', '네때어 다녀요말도 제법 잘하구요너무너무 예쁠데 힘들어요', '진짜 뿌듯할 거 같어', '상품권을 갇으면 어자 쐬는거 좋을까요', '너무 좋거라고요 ', '올여름에 엄청 더웠자나']


In [380]:
print(len(correct_text), len(typo_text), len(output_text_list))

200 200 200


In [383]:
for idx, correct in enumerate(correct_text):
    correct = correct.replace(" ", "")
    output_text = output_text_list[idx].replace(" ", "")
    if correct == output_text:
        print(idx)

In [353]:
a = [1,2,4,4]
a.index(max(a))

2